In [3]:
import os
import sys
import traci
import pandas as pd
import numpy as np
from itertools import product
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from xml.dom import minidom

In [4]:
def extract_data_from_xml(xml_file, file_number):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    all_data =[]

    for emis_elem in root.findall('.//timestep'):
        time = float(emis_elem.get('time'))
        if 5401 <= time <= 7200:
            for vehicle in emis_elem.findall('vehicle'):
                data = {'file_number': file_number,'time':time}
                for attr in vehicle.attrib:
                    data[attr] = str(vehicle.get(attr))
                all_data.append(data)

    return all_data

In [5]:
# doing the summary of emission in two part due to large file size
xml_directory='emission/first20'
all_data = []

for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(xml_directory, xml_file)
        file_number = int(xml_file.split('_')[-1].split('.')[0])
        all_data.extend(extract_data_from_xml(xml_path, file_number))

df = pd.DataFrame(all_data)
print(df.head(5))

In [6]:
df = df.sort_values(by=['file_number','time']) 

In [7]:
df.to_csv("./IDM_allcar_time_resultpart1.csv")

In [8]:
#print(df.head(5))

In [9]:
xml_directory='/emission/second20'
all_data = []

for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(xml_directory, xml_file)
        file_number = int(xml_file.split('_')[-1].split('.')[0])
        all_data.extend(extract_data_from_xml(xml_path, file_number))

df = pd.DataFrame(all_data)
df = df.sort_values(by=['file_number','time']) 
df.to_csv("./IDM_allcar_time_resultpart2.csv")

In [10]:
print(df.head())

In [11]:
columns_index = df.columns
print(columns_index)
cols_to_convert = ['CO2', 'CO', 'HC', 'NOx', 'PMx', 'fuel','waiting']

In [12]:
csv_file = './IDM_allcar_time_resultpart1.csv'
data1 =pd.read_csv(csv_file)
csv_file2 = './IDM_allcar_time_resultpart2.csv'
data2 =pd.read_csv(csv_file2)
data  =pd.concat([data1, data2], axis=0, ignore_index=True)
for col in cols_to_convert:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [13]:
result = data.groupby('file_number').agg(
    unique_ids=('id', 'nunique'),
    total_CO2=('CO2', 'sum'),
    total_CO=('CO', 'sum'),
    total_HC=('HC', 'sum'),
    total_NOX=('NOx','sum'),
    total_PMX=('PMx', 'sum'),
    total_fuel=('fuel', 'sum'),
    total_waiting=('waiting','sum')
).reset_index()

In [1]:
#print(result.head(5))

In [14]:
result.to_csv("./emission_result.csv")

In [15]:
def extract_data_from_xml(xml_file, file_number):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    all_data = []

    for edge_elem in root.findall('.//edge'):
        edge_id = edge_elem.get('id')
        data = {'file_number': file_number, 'id': (edge_id)}
        for attr in edge_elem.attrib:
            data[attr] = (edge_elem.get(attr))

        all_data.append(data)

    return all_data
xml_directory='./edge_emission'
all_data = []

for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(xml_directory, xml_file)
        file_number = int(xml_file.split('_')[-1].split('.')[0])
        all_data.extend(extract_data_from_xml(xml_path, file_number))
    df = pd.DataFrame(all_data)

cols_to_convert = ['sampledSeconds', 'CO_abs', 'CO2_abs', 'HC_abs',
       'PMx_abs', 'NOx_abs', 'fuel_abs', 'electricity_abs', 'CO_normed',
       'CO2_normed', 'HC_normed', 'PMx_normed', 'NOx_normed', 'fuel_normed',
       'electricity_normed', 'traveltime', 'CO_perVeh', 'CO2_perVeh',
       'HC_perVeh', 'PMx_perVeh', 'NOx_perVeh', 'fuel_perVeh',
       'electricity_perVeh']

# Convert specified columns to numeric
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

total_emission_13ms = df.groupby('file_number').agg({'CO_abs':'sum',
                                                     'CO2_abs':'sum',
                                                     'HC_abs':'sum',
                                                     'PMx_abs':'sum',
                                                     'NOx_abs':'sum',
                                                     'fuel_abs':'sum'}).reset_index()
total_emission_13ms.to_csv("./edge_emission_result.csv")

In [5]:
#####Tital simulation time emission and fuel consumtion
#alternative approach from edge_emission
def extract_data_from_xml(xml_file, file_number):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    all_data = []

    for tripinfo in root.findall('.//tripinfo'):
        trip_id = tripinfo.get('id')
        data = {'file_number': file_number, 'id': (trip_id)}
        emissions = tripinfo.find('emissions')
        if emissions is not None:
            for attr in emissions.attrib:
                data[attr] = emissions.get(attr)
        
        all_data.append(data)

    return all_data

In [6]:
xml_directory='./tripinfos'
all_data = []

for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(xml_directory, xml_file)
        file_number = int(xml_file.split('_')[-1].split('.')[0])
        all_data.extend(extract_data_from_xml(xml_path, file_number))
    df = pd.DataFrame(all_data)



In [7]:
columns_index = df.columns
print(columns_index)

Index(['file_number', 'id', 'CO_abs', 'CO2_abs', 'HC_abs', 'PMx_abs',
       'NOx_abs', 'fuel_abs', 'electricity_abs'],
      dtype='object')


In [10]:
cols_to_convert = ['file_number','CO_abs', 'CO2_abs', 'HC_abs', 'PMx_abs',
       'NOx_abs', 'fuel_abs', 'electricity_abs']
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

total_emission_13ms = df.groupby('file_number').agg({'CO_abs':'sum',
                                                     'CO2_abs':'sum',
                                                     'HC_abs':'sum',
                                                     'PMx_abs':'sum',
                                                     'NOx_abs':'sum',
                                                     'fuel_abs':'sum','id':'nunique'
                                                    }).reset_index()
total_emission_13ms.to_csv("emission_resultfullsim.csv")


In [2]:
############Extract from trip info
def extract_data_from_xml(xml_file, file_number):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    all_data = []
    # Iterate through the XML tree
    for tripinfo in root.findall('tripinfo'):
        time = float(tripinfo.get('depart'))
        if 5401 <= time <= 7200:  
            data = {'file_number': file_number, 'time': time}
            for attr in tripinfo.attrib:
                data[attr] = tripinfo.get(attr)  
            emissions = tripinfo.find('emissions')
            if emissions is not None:
                for attr in emissions.attrib:
                    data[attr] = emissions.get(attr) 
            all_data.append(data)
    return all_data
xml_directory='./tripinfos'

all_data = []

for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(xml_directory, xml_file)
        file_number = int(xml_file.split('_')[-1].split('.')[0])
        all_data.extend(extract_data_from_xml(xml_path, file_number))

df = pd.DataFrame(all_data)
#print(df.head(5))
df=df.sort_values(by=['file_number','time'])
#print(df)
cols_to_convert = ['CO2_abs', 'CO_abs', 'HC_abs', 'PMx_abs', 'NOx_abs', 'fuel_abs']
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [18]:
total_emission = df.groupby('file_number').agg({'CO2_abs':'sum',
                                                'CO_abs':'sum',
                                                'HC_abs':'sum',
                                                'PMx_abs':'sum',
                                                'NOx_abs':'sum',
                                                'fuel_abs':'sum',
                                                'id':'nunique'}).reset_index()
total_emission.to_csv("tripinfo_resultschooltime.csv")